```{contents}
```
## **Router Node in LangGraph**

A **Router Node** is a control component in LangGraph that **dynamically determines the next execution path** of a graph based on the current state.
It enables **conditional branching, decision-making, and adaptive workflows**, making LangGraph suitable for complex agent systems.

---

### **1. Intuition**

In static pipelines, the execution path is fixed.
In LangGraph, execution is **state-driven**.

A Router Node acts as the **decision function**:

> **State → Decision → Next Node**

This turns the graph into a **programmable control system**.

---

### **2. Where the Router Fits in a Graph**

```
[Node A] → [Router] ──→ [Node B]
                ├──→ [Node C]
                └──→ [Node D]
```

The router selects **exactly one outgoing edge** at runtime.

---

### **3. Formal Definition**

A Router Node is implemented using:

```python
builder.add_conditional_edges(
    source_node,
    routing_function,
    path_map
)
```

Where:

* `routing_function(state) → key`
* `key ∈ path_map`
* `path_map[key] → next_node`

---

### **4. Basic Example**

### **State**

```python
class State(TypedDict):
    query: str
    intent: str
```

### **Router Logic**

```python
def route_by_intent(state):
    if "price" in state["query"]:
        return "pricing"
    if "error" in state["query"]:
        return "support"
    return "general"
```

### **Graph Wiring**

```python
builder.add_node("router", route_by_intent)

builder.add_conditional_edges(
    "router",
    route_by_intent,
    {
        "pricing": "pricing_node",
        "support": "support_node",
        "general": "general_node"
    }
)
```

---

### **5. LLM-Powered Router**

The router itself may call an LLM.

```python
def llm_router(state):
    prompt = f"Classify the intent: {state['query']}"
    label = llm.invoke(prompt)
    return label.lower()
```

Used for:

* Intent classification
* Task decomposition
* Tool selection
* Agent delegation

---

### **6. Common Routing Patterns**

| Pattern            | Purpose                  |
| ------------------ | ------------------------ |
| Intent Router      | NLP-based decision       |
| Tool Router        | Select best tool         |
| Agent Router       | Assign specialized agent |
| Error Router       | Recovery logic           |
| Loop Router        | Control cyclic flow      |
| Termination Router | Stop condition           |

---

### **7. Router in Cyclic Graphs**

Routers are essential for loop control.

```python
def loop_router(state):
    if state["done"]:
        return END
    return "continue"
```

---

### **8. Production Considerations**

| Aspect        | Practice                       |
| ------------- | ------------------------------ |
| Determinism   | Keep routing logic predictable |
| Fallback      | Always include default path    |
| Safety        | Validate returned route        |
| Observability | Log routing decisions          |
| Latency       | Cache expensive routing calls  |
| Testing       | Unit-test routing functions    |

---

### **9. Mental Model**

A Router Node transforms LangGraph from a **workflow engine** into a **decision engine**.

> Without routers → fixed pipelines
> With routers → intelligent systems

---

### **10. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END

class State(TypedDict):
    x: int

def step(state):
    return {"x": state["x"] + 1}

def router(state):
    return "loop" if state["x"] < 5 else "stop"

builder = StateGraph(State)
builder.add_node("step", step)
builder.set_entry_point("step")

builder.add_conditional_edges(
    "step",
    router,
    {
        "loop": "step",
        "stop": END
    }
)

graph = builder.compile()
print(graph.invoke({"x": 0}))
```

---

### **Summary**

The Router Node provides:

* **Dynamic control flow**
* **Adaptive decision-making**
* **Autonomous behavior**
* **Safe termination**
* **Scalable agent coordination**

It is the **central intelligence switch** of any serious LangGraph system.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    x: int

def step_node(state: State):
    print(f"Step: {state['x']}")
    return {"x": state["x"] + 1}

def router(state: State):
    if state["x"] < 5:
        return "loop"
    return "stop"



In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("step", step_node)

builder.set_entry_point("step")

builder.add_conditional_edges(
    "step",
    router,
    {
        "loop": "step",
        "stop": END
    }
)


In [3]:
graph = builder.compile()
result = graph.invoke({"x": 0})
print("Final State:", result)


Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Final State: {'x': 5}
